노래가사 크롤링
============

- 가수별 인기순 노래가사 수집 
- 크롤링 : Keyword, Artist, Title, Like, Lyrics, Datetime(발매일), Genre

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import time
from tqdm import tqdm, trange 
%matplotlib inline

In [0]:
# 그래프에서 한글표현을 위해 폰트를 설치합니다.
%config InlineBackend.figure_format = 'retina'

!apt -qq -y install fonts-nanum > /dev/null

In [0]:
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

In [0]:
# 기본 글꼴 변경
import matplotlib as mpl
mpl.font_manager._rebuild()
mpl.pyplot.rc('font', family='NanumBarunGothic')

In [0]:
#url sort = p 인기순 

def get_suggestion_list(singers, pages):
    suggestion_list = []
    suggestion_album = []
    singer_list = [] 
    for singer in singers: 
        for page in range(pages): 
            base_url = requests.get("https://music.bugs.co.kr/search/track?q=" + singer+'&flac_only=false&target=ARTIST_TRACK_ALBUM&page='+str(page+1)+'&sort=P')
            if base_url.status_code == 200 : 
                soup = BeautifulSoup(base_url.text, 'html.parser', from_encoding="utf8")
                tags = soup.select('#DEFAULT0 > table > tbody')[0].find_all('tr')
                for tag in tags:
                    suggestion_list.append('https://music.bugs.co.kr/track/'+tag['trackid'])
                    suggestion_album.append('https://music.bugs.co.kr/album/'+tag['albumid'])
                    singer_list.append(singer)
            
    return suggestion_list, suggestion_album, singer_list

In [0]:
def get_suggestion_content(url_list):
    base_url2 = requests.get(url_list)
    soup2 = BeautifulSoup(base_url2.text, 'html.parser', from_encoding="utf8")
    lyrics_list = [] 
    tags1 = soup2.select('#container > header > div')[0].find_all('h1')
    title = tags1[0].get_text(strip=True).replace('(inst.)','')
    tags2 = soup2.select('#container > section.sectionPadding.contents.lyrics > div > div')[0].find_all('xmp')
    lyrics = tags2[0].get_text(strip=True).replace('\r\n',' ').replace('\r',' ').replace('\n',' ')
    #container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody > tr:nth-child(1) > th
    tags3 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody')[0].find_all('td')
    artist = tags3[0].get_text(strip=True)
    
    #container > section.sectionPadding.summaryInfo.summaryTrack > div > div.etcInfo > span > a > span > em
    tags4 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.etcInfo > span > a > span')[0].find_all('em')
    like = tags4[0].get_text(strip=True).replace(',','')

    
    lyrics_list.append(artist)
    lyrics_list.append(title)
    lyrics_list.append(like)
    lyrics_list.append(lyrics)
        
    return lyrics_list 

In [0]:
def get_suggestion_album(url_list):
    base_url2 = requests.get(url_list)
    soup2 = BeautifulSoup(base_url2.text, 'html.parser', from_encoding="utf8")
    info = []
    tags1 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryAlbum > div > div.basicInfo > table')[0].find_all('time')
    datetime = tags1[0].get_text(strip=True)
    #container > section.sectionPadding.summaryInfo.summaryAlbum > div > div.basicInfo > table > tbody > tr:nth-child(4) > td
    tags2 = soup2.select('#container > section.sectionPadding.summaryInfo.summaryAlbum > div > div.basicInfo > table > tbody')[0].find_all('td')
    genre = tags2[4].get_text(strip=True)
       
    info.append(datetime)
    info.append(genre)
        
    return info

In [0]:
def get_data(singer, pages): 
    data1= []
    data2= [] 
    
    suggestion_list = get_suggestion_list(singer,pages)[0]
    info_list = get_suggestion_list(singer,pages)[1]  
    singer_list = get_suggestion_list(singer,pages)[2]
    
    for i, suggestion in tqdm(enumerate(suggestion_list)):
        s = get_suggestion_content(suggestion)
        data1.append(s)
    
    for i, info in tqdm(enumerate(info_list)):
        s = get_suggestion_album(info)
        data2.append(s)
        
    data1 = pd.DataFrame(data1, columns=["Artist","Title","Like","Lyrics"])
    data2 = pd.DataFrame(data2, columns=["Datetime","Genre"])
    data3 = pd.DataFrame(singer_list, columns=["Keyword"])
    data = pd.concat([data3, data1, data2], axis = 1)
    
    return data 

In [0]:
singer = ["아이유"]
data = get_data(singer, 1) #페이지1에 곡 50개 

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
50it [01:07,  1.32s/it]
50it [01:01,  1.28s/it]


동일 곡 중복제거 

In [0]:
#data = data.drop_duplicates()

In [0]:
data.head(3)

데이터 전처리

Quiz
====
1. Genre별 분석 
 - 가장 많은 노래가 발매 된 장르는 ? 
 - 가장 많은 Like를 받은 장르는 ? 


2. 타가수와 콜라보여부 
 - 타가수와 콜라보를 진행한 노래와 아닌 노래 중 더 높은 Like를 받은 노래는? 
 

3. 발매일 
 - 가장 많은 노래가 발매 된 연월은 ? 


4. 노래가사에서 가장 많이 언급된 단어는?  


5. TOP1 언급 단어와 유사도가 가장 높은 노래는? 


In [0]:
#문자형을 숫자형으로 변경 
data['Like'] = data['Like'].astype(int)
#문자형을 데이터타입으로 변경 
data['Datetime'] = pd.to_datetime(data['Datetime'])
data['Date-month'] = data['Datetime'].dt.year.astype(str) +"-"+ data['Datetime'].dt.month.astype(str)
data['Group'] = data['Artist'] != '아이유(IU)' 
data['Group'] = data['Group'].astype(str)

In [0]:
data.info()

Q1) 노래 Genre별 분석 
-------------------------------

In [0]:
figure,(ax1, ax2) = plt.subplots(1, 2)
figure.set_size_inches(15,10)
# 장르별 노래수
sns.countplot(data=data, y="Genre", ax=ax1)
# 장르별 like수
sns.barplot(data=data, x="Like", y="Genre", ax=ax2)

Q2) 타가수와 콜라보 여부에 따른 선호도 비교 
-------------------------------------------------------------

In [0]:
figure,(ax1, ax2) = plt.subplots(1, 2)
figure.set_size_inches(15,5)
# 듀엣 / 솔로 
sns.countplot(data=data, y="Group", ax=ax1)
# 장르별 like수
sns.barplot(data=data, x="Like", y="Group", ax=ax2)

Q3) 기준년월별 노래 발매현황
--------------------------------------------------------

In [0]:
plt.figure(figsize=(18,5))
plt.title('월별 발매 수')
sns.countplot(data=data.sort_values(by="Date-month", ascending=True), x="Date-month")

In [0]:
#colab에서 실행 시 konlpy 설치
!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

Q4) 노래가사 언급 단어 분석
--------------------------------------

In [0]:
from konlpy.tag import Kkma
tagger = Kkma()

In [0]:
stop_words = "내 나 너 거 날 때 수 뭐 위 오 번 둘 안 널 가 이 은 인"
stop_words = stop_words.split(' ')
stop_words

In [0]:
from collections import Counter
def get_wordcloud(keyword, value, columns, n):
    lyrics = list(data.loc[data[keyword] == value, columns])
    word_list = []
    for post in lyrics:
        for noun in tagger.nouns(post):
            if noun not in stop_words:
                word_list.append(noun)
    num_top_nouns = n
    nouns_counter = Counter(word_list)
    top_nouns = dict(nouns_counter.most_common(num_top_nouns))
    
    return top_nouns 

In [0]:
top_nouns_lyrics = get_wordcloud("Keyword","아이유","Lyrics",30)

In [0]:
plt.figure(figsize=(20,5))
sns.barplot(y=list(top_nouns_lyrics.values()), x=list(top_nouns_lyrics.keys()))

[Wordcloud 표현] 


1) 노래가사 언급 TOP30 기준 

In [0]:
from wordcloud import WordCloud

In [0]:
wc = WordCloud(background_color="white", font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf')

In [0]:
wc.generate_from_frequencies(top_nouns_lyrics)
plt.axis('off')
plt.title("아이유 노래가사 Wordcloud")
plt.imshow(wc)

2) 노래가사 (Genre=팝) TOP 30 기준

In [0]:
top_nouns = get_wordcloud("Genre","팝","Lyrics",30)
wc.generate_from_frequencies(top_nouns)
plt.axis('off')
plt.title("발라드 Wordcloud")
plt.imshow(wc)

Q5) 언급량 TOP1 단어와 유사도가 가장 높은 노래는?
-----------------------------------------------------------------------

In [0]:
top_nouns = get_wordcloud("Keyword","아이유","Lyrics",20)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidv = TfidfVectorizer().fit(top_nouns)

In [0]:
features = tfidv.get_feature_names()

In [0]:
tfidv_data = pd.DataFrame(tfidv.transform(data['Lyrics']).toarray(), columns = features)

In [0]:
data_f = pd.concat([data[['Artist','Title']], tfidv_data], axis =1)

In [0]:
data_f.head(10)

In [0]:
data_f.sort_values(by='사랑', ascending = False).head(5)

In [0]:
list(data.loc[data['Keyword'] == '아이유', 'Lyrics'])[5]